In [ ]:
!pip install efficientnet
!pip install keras-radam
!pip install adabound
!pip install keras-adabound

In [ ]:
!pip install tensorflow==1.14
import tensorflow as tf

tf.__version__

'1.14.0'

In [ ]:
import tensorflow as tf
import shutil
import numpy as np
import pandas as pd
import cv2
import os
import time
import matplotlib.pyplot as plt
import glob
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten, ReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, SGD
import efficientnet.keras as enet
from sklearn.model_selection import train_test_split
from keras_radam import RAdam
from keras_adabound import AdaBound
# from adabound import AdaBound


In [ ]:
def idx_generator(labels1):
    ''' Condition on labels '''
    idx_0 = np.where(labels1 == 0)[0]
    idx_1 = np.where(labels1 == 1)[0]
    malignant_idx = np.random.choice(idx_0, len(idx_1), replace=False)
    benign_idx = np.random.choice(idx_1, len(idx_1), replace=False)

    ''' Concat data '''
    labels_idx1 = np.concatenate([malignant_idx, benign_idx])
    ''' Random labels '''
    labels_idx1 = np.random.choice(labels_idx1, len(labels_idx1), replace=False)
    return labels_idx1


def dataset_equalizer(dataset11, labels11):
    ''' Choose malignant and benign data '''
    # dataset11, labels11 = data_loader(t00)
    labels_idx11 = idx_generator(labels11)
    labels11 = labels11[labels_idx11]
    dataset11 = dataset11[labels_idx11]
    return dataset11, labels11




In [ ]:
''' Seed for reproducible results '''
# Seed value
# Apparently you may use different seed values at each stage
seed_value = 0
# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os

os.environ['PYTHONHASHSEED'] = str(seed_value)
# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random

random.seed(seed_value)
# 3. Set the `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)
# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_random_seed(seed_value)
# for later versions:
# tf.compat.v1.set_random_seed(seed_value)
# 5. Configure a new global `tensorflow` session
from keras import backend as K

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)
# for later versions:
# session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
# tf.compat.v1.keras.backend.set_session(sess)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
''' Read labels '''
all_files = glob.glob("drive/My Drive/kaggle_ddsm/low_res_200_200/*.png")
# print(all_files)
labels = np.zeros(len(all_files), dtype=np.float)
for i in range(len(all_files)):
    files = all_files[i].split('/')[4]
    # print(files)
    labels[i] = int(files.split('_')[4])
print(labels)





[0. 0. 0. ... 0. 1. 0.]


In [ ]:
''' Read images '''
dataset = []
for i in range(len(all_files)):
    # image = plt.imread(all_files[i])
    image = cv2.imread(all_files[i], cv2.IMREAD_GRAYSCALE)
    ''' Normalize images '''
    # image = image.astype('float32')
    # image /= np.float(np.max(image))
    dataset.append(image)
dataset = np.array(dataset, dtype=np.float)
print(dataset.shape)

(11177, 120, 120)


In [ ]:
''' Equalize dataset malignant and benign labels '''
dataset, labels = dataset_equalizer(dataset, labels)
print(dataset.shape)
print(labels.shape)
print(labels)

''' Normalize images '''
##################################
# Normalization task is done in reading part
##################################
print('Normalization')
dataset = dataset.astype('float32')
print(np.max(dataset))
print(np.min(dataset))
dataset /= np.float(255)
print(np.max(dataset))
print(np.min(dataset))



(2958, 120, 120)
(2958,)
[0. 0. 1. ... 1. 1. 0.]
Normalization
255.0
0.0
1.0
0.0


In [ ]:
''' Image channels compatible with the network '''
# 3 Channel image:
dataset = np.stack((dataset,) * 3, axis=-1)
x_train_1, x_test_1, y_train1, y_test1 = train_test_split(dataset, labels, test_size=0.001)  # , shuffle=True)
# x_train_1 = dataset
# y_train1 = labels

''' Categorical labels '''
from keras.utils import to_categorical

y_train1 = to_categorical(y_train1, num_classes=len(np.unique(labels)))
y_test1 = to_categorical(y_test1, num_classes=len(np.unique(labels)))



In [ ]:
''' Efficient Net codes: '''

print(dataset.shape)
# print(y_train1)

''' Define Swish activation function '''
# Swish definition
from keras.backend import sigmoid


class SwishActivation(Activation):

    def __init__(self, activation, **kwargs):
        super(SwishActivation, self).__init__(activation, **kwargs)
        self.__name__ = 'swish_act'


def swish_act(x, beta=1):
    return (x * sigmoid(beta * x))


from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation

get_custom_objects().update({'swish_act': SwishActivation(swish_act)})



(2958, 120, 120, 3)


In [ ]:
''' Initialization '''
# x_train_1 = x_train_1[..., np.newaxis]
# # x_train_2 = x_train_2[..., np.newaxis]
# x_test_1 = x_test_1[..., np.newaxis]
# # x_test_2 = x_test_2[..., np.newaxis]
# print(y_train1.shape)
# print(y_test1.shape)
input_shape = x_train_1.shape[1:]
print(input_shape)

# loading B0 pre-trained on ImageNet without final aka fiature extractor
model = enet.EfficientNetB0(include_top=False, pooling='avg', input_shape=(120, 120, 3), weights='imagenet')

# # building 2 fully connected layer
x = model.output

# x = BatchNormalization()(x)
# x = Dropout(0.2)(x)
#
# x = Dense(512)(x)
# x = BatchNormalization()(x)
# x = Activation(swish_act)(x)
# x = Dropout(0.2)(x)
#
# x = Dense(128)(x)
# x = BatchNormalization()(x)
# x = Activation(swish_act)(x)

# output layer
predictions = Dense(2, activation="softmax")(x)

model_final = Model(inputs=model.input, outputs=predictions)

model_final.summary()



(120, 120, 3)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 60, 60, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 60, 60, 32)   128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 60, 60, 32)   0           stem_bn[0][0]                    
______________________________________________________________________________

In [ ]:
''' Depict model '''
# model_final.summary()
# from keras.utils import plot_model
# import matplotlib.pyplot as plt
# import os
# import cv2
#
# os.environ["PATH"] += os.pathsep + 'D:\\university\\term_4\\my_libs\\release\\bin'
# plot_model(model_final, to_file='model_single.png', show_shapes=True, show_layer_names=False)
# image = cv2.imread('model_single.png')
# plt.imshow(image)
# plt.show()



' Depict model '

In [ ]:
''' Plot online results (Live results)'''
from IPython.display import clear_output


class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acces = []
        self.val_acces = []

        self.fig = plt.figure()

        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acces.append(logs.get('acc'))
        self.val_acces.append(logs.get('val_acc'))
        self.i += 1

        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.grid(True)
        plt.show()

        ''' Validation Accuracy plot '''
        # plt.plot(self.x, self.acces, label="acc")
        # plt.plot(self.x, self.val_acces, label="val_acc")
        # plt.legend()
        # plt.grid(True)
        # plt.show()


plot_losses = PlotLosses()



In [ ]:
# !pip install adabound-keras
from keras.optimizers import Optimizer
from keras.legacy import interfaces
from keras import backend as K
import tensorflow as tf


class AdaBound_class(Optimizer):
    def __init__(self, lr=0.001,
                 beta_1=0.9, beta_2=0.999,
                 gamma=0.001,
                 final_lr=0.1,
                 epsilon=None,
                 decay=0.,
                 amsbound=False,
                 **kwargs):
        super().__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_1, name='beta_2')
            self.gamma = K.variable(gamma, name='gamma')
            self.final_lr = K.variable(final_lr, name='final_lr')
            self.decay = K.variable(decay, name='decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.amsbound = amsbound

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations,
                                                      K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]

        if self.amsbound:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros((1, 1)) for _ in params]

        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            lower_bound_t = self.final_lr * (1 - 1 / self.gamma * t + 1)
            upper_bound_t = self.final_lr * (1 + 1 / self.gamma * t)

            if self.amsbound:
                vhat_t = K.maximum(vhat, v_t)
                # p_t = p - K.clip(lr_t * (K.sqrt(vhat_t) + self.epsilon),
                #                  lower_bound_t,
                #                  upper_bound_t)
                p_t = p - tf.clip_by_value(
                    lr_t * (K.sqrt(vhat_t) + self.epsilon),
                    lower_bound_t,
                    upper_bound_t)
            else:
                # p_t = p - K.clip(lr_t * (K.sqrt(v_t) + self.epsilon),
                #                  lower_bound_t,
                #                  upper_bound_t)
                p_t = p - tf.clip_by_value(
                    lr_t * (K.sqrt(v_t) + self.epsilon),
                    lower_bound_t,
                    upper_bound_t)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))

        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'gamma': float(K.get_value(self.gamma)),
                  'final_lr': float(K.get_value(self.final_lr)),
                  'decay': float(K.get_value(self.decay)),
                  'epsilon': self.epsilon,
                  'amsbound': self.amsbound}
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizers import Optimizer
from tensorflow.python.ops import state_ops
from tensorflow.python.ops import math_ops


class AdaBound1(Optimizer):
    """AdaBound optimizer.
    Default parameters follow those provided in the original paper.
    Arguments:
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        final_lr: float >= 0. Final learning rate.
        gamma: float >= 0. Convergence speed of the bound functions.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        amsbound: boolean. Whether to use the AMSBound variant of this algorithm
            from the paper "Adaptive Gradient Methods with Dynamic Bound of Learning Rate".
    """

    def __init__(self,
                 lr=0.001,
                 beta_1=0.9,
                 beta_2=0.999,
                 final_lr=0.1,
                 gamma=0.001,
                 epsilon=1e-8,
                 decay=0.,
                 amsbound=False,
                 **kwargs):
        super(AdaBound, self).all(**kwargs)#.__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.final_lr = K.variable(final_lr, name='final_lr')
            self.gamma = K.variable(gamma, name='gamma')
            self.decay = K.variable(decay, name='decay')
            self.amsbound = K.variable(amsbound, name='amsbound')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = K.variable(epsilon)
        self.initial_decay = decay
        self.base_lr = lr

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [state_ops.assign_add(self.iterations, 1)]

        lr = self.lr
        if self.initial_decay > 0:
            lr = lr * (
                    1. / (1. + self.decay * math_ops.cast(self.iterations,
                                                          K.dtype(self.decay))))

        t = math_ops.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (
                K.sqrt(1. - math_ops.pow(self.beta_2, t)) /
                (1. - math_ops.pow(self.beta_1, t)))

        final_lr = self.final_lr * lr / self.base_lr
        lower_bound = final_lr * (1. - 1. / (self.gamma * t + 1))
        upper_bound = final_lr * (1. + 1. / (self.gamma * t))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsbound:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * math_ops.square(g)
            if self.amsbound:
                vhat_t = math_ops.maximum(vhat, v_t)
                p_t = p - m_t * K.clip(lr_t / (K.sqrt(vhat_t) + self.epsilon), lower_bound, upper_bound)
                self.updates.append(state_ops.assign(vhat, vhat_t))
            else:
                p_t = p - m_t * K.clip(lr_t / (K.sqrt(v_t) + self.epsilon), lower_bound, upper_bound)

            self.updates.append(state_ops.assign(m, m_t))
            self.updates.append(state_ops.assign(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(state_ops.assign(p, new_p))
        return self.updates

    def get_config(self):
        config = {
            'lr': float(K.get_value(self.lr)),
            'beta1': float(K.get_value(self.beta_1)),
            'beta2': float(K.get_value(self.beta_2)),
            'final_lr': float(K.get_value(self.final_lr)),
            'gamma': float(K.get_value(self.gamma)),
            'decay': float(K.get_value(self.decay)),
            'epsilon': self.epsilon,
            'amsbound': self.amsbound
        }
        base_config = super(AdaBound, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
''' Train model '''
# model compilation
# Memory check
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.log_device_placement = True
# sess = tf.Session(config=config)
# set_session(sess)
# from keras_adabound import AdaBound
# from adabound_keras.adabound import AdaBound

# from adabound import AdaBound
# import adabound

sgd = SGD(lr=0.0007, decay=1e-6, momentum=0.8, nesterov=False)
# optm = adabound.AdaBound(model.parameters(), lr=1e-3, final_lr=0.1)

# optm = AdaBound(lr=1e-03,
#                 final_lr=0.1,
#                 gamma=1e-03,
#                 weight_decay=0.,
#                 # amsbound=False,
#                 )

# optm = AdaBound(lr=8*1e-04,
#                 # param=None,
#                 final_lr=0.1,
#                 gamma=1e-03)

a = AdaBound1()
optm = a(lr=0.001, beta1=0.9, beta2=0.999, final_lr=0.1, gamma=1e-3, epsilon=None, weight_decay=0, amsbound=False)

run_opts = tf.RunOptions(report_tensor_allocations_upon_oom=True)
model_final.compile(loss='binary_crossentropy',
                    optimizer=optm,
                    metrics=['accuracy'])

mcp_save = ModelCheckpoint('EnetB0_CIFAR10_TL.h5', save_best_only=True, monitor='val_accuracy')
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=2, verbose=1, )

# print("Training....")
history = model_final.fit(x_train_1, y_train1,
                          batch_size=16,
                          epochs=30,
                          validation_split=0.1,
                          callbacks=[plot_losses, mcp_save, reduce_lr],
                          # shuffle=True,
                          verbose=1)
# history = model.fit(x_train_1, y_train1,
#                     validation_data=(x_test_1, y_test1),
#                     callbacks=[plot_losses],
#                     epochs=10, batch_size=4, shuffle=True)



TypeError: ignored

In [ ]:
''' Results '''
# eval = model.evaluate(x=x_test_1, y=y_test1)

print(history.history)

# print(eval)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.grid(True)
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.grid(True)
plt.legend(['Train', 'Test'], loc='upper right')
plt.show()

evaluation = model_final.evaluate(x=x_test_1, y=y_test1)
print("test loss, test acc:", evaluation)

print(history.history['val_accuracy'])
print(np.max(history.history['val_accuracy']))